# 0. Import Libraries

In [70]:
import pandas as pd
import requests
import pyreadr

# 1. Extracting Data

### 1. 1. Section Enrollment Data - After 2017

In [71]:
# Load Scraped data
section_final_df = pd.read_csv("Extracted_files/courses_processed.csv", low_memory=False)

Combined 205132 rows from 17 files


### 1. 2. Grades Data - After 2017

In [72]:
gpa_base_url = "https://raw.githubusercontent.com/wadefagen/datasets/main/gpa/raw/"

# Files from Spring and Fall (2017–2024) only
gpa_files = []
for year in range(2017, 2025):
    gpa_files.extend([
        f"sp{year}.csv",
        f"fa{year}.csv"
    ])

# To hold all loaded data
final_gpa_df = []

for file in gpa_files:
    url = gpa_base_url + file
    try:
        # Try UTF-8 first
        try:
            gpa_df = pd.read_csv(url, encoding="utf-8")
        except UnicodeDecodeError:
            gpa_df = pd.read_csv(url, encoding="ISO-8859-1")

        # Extract term and year
        name = file.replace(".csv", "")
        raw_term = name[:2].upper()
        term_map = {"SP": "SPRING", "FA": "FALL"}
        term = term_map.get(raw_term, raw_term)
        year = int(name[2:])

        # Filter here: only SPRING and FALL after 2017
        if year > 2017 and term in ["SPRING", "FALL"]:
            gpa_df["Year"] = year
            gpa_df["Term"] = term
            final_gpa_df.append(gpa_df)

    except Exception as e:
        print(f"Failed to load {file}: {e}")

# Combine all filtered data
all_gpa_df = pd.concat(final_gpa_df, ignore_index=True)

# Drop rows with missing Course values in GPA
all_gpa_df = all_gpa_df[all_gpa_df["Course"].notnull()]

# Convert Course (GPA) to string and remove decimal
all_gpa_df["Course"] = all_gpa_df["Course"].astype(int).astype(str)

print(f"Loaded {len(final_gpa_df)} files with {len(all_gpa_df)} rows (Spring/Fall, after 2017).")
all_gpa_df.to_csv("Extracted_files/1_filtered_gpa_sp_fa_after_2017.csv", index=False)


Loaded 14 files with 19284 rows (Spring/Fall, after 2017).


### 1. 3. Merge Two Sources

In [115]:
# Ensure 'Course' in GPA is str and stripped to match 'Number'
all_gpa_df["Course"] = all_gpa_df["Course"].astype(str).str.strip()

# Ensure 'Number' in section data is also str and stripped
section_final_df["Number"] = section_final_df["Number"].astype(str).str.strip()

# Ensure CRN is the same type in both (int or str), safest to cast to str
all_gpa_df["CRN"] = all_gpa_df["CRN"].astype(str).str.strip()
section_final_df["CRN"] = section_final_df["CRN"].astype(str).str.strip()

# Now perform the join on the required columns
merged_df = pd.merge(
    section_final_df,
    all_gpa_df,
    how="inner",
    left_on=["Year", "Term", "CRN"],
    right_on=["Year", "Term", "CRN"]
)

before = merged_df.shape[0]

### 1. 4. Exporting Merged DB for Profiling

In [116]:
# Preview and save
print(f"Merged dataset has {len(merged_df)} rows.")
merged_df.to_csv("Extracted_files/2_merged_uiuc_courses_gpa.csv", index=False)

Merged dataset has 20042 rows.


# 2. Data Cleaning 

### 2. 1. Checking Common Columns

In [117]:
subject_check = merged_df['Subject_x'].equals(merged_df['Subject_y'])
print("subject: ", subject_check)

course_name_check = merged_df['Name'].equals(merged_df['Course Title'])
print("course_name_check: ", course_name_check)

course_no_check = merged_df['Number'].equals(merged_df['Course'])
print("course_no_check: ", course_no_check)

section_check = merged_df['Section_x'].equals(merged_df['Section_y'])
print("section: ", section_check)

section_status_check = merged_df['Section Status'].equals(merged_df['Status Code'])
print("section status: ", section_status_check)

subject:  True
course_name_check:  False
course_no_check:  True
section:  False
section status:  True


1. **Subjects:** The subject_x and subject_y columns contain identical information. One of them has been dropped to avoid redundancy.

2. **Name vs. Course Title:** Although not textually identical, the Name and Course Title columns represent the same information. Minor differences are due to abbreviations or formatting. These have been reconciled as equivalent.

3. **Course Numbers:** The course numbers are consistent across datasets and require no transformation.

4. **Sections:** While the section_x and section_y columns are not identical, they are complementary—if one is missing, the other typically contains the correct value. A unified section column has been created by taking the non-null value from either column.

5. **Section Status:** The SectionStatus_x and SectionStatus_y columns are identical. One has been dropped to remove duplication.

6. **Credit Hours:** Both CreditHours and GPA Credit Hours columns are retained. CreditHours refers to the total available credits for a course, whereas GPA Credit Hours reflects the credits applicable to GPA calculations. These serve distinct analytical purposes.

7. **Part of Term:** The PartOfTerm column contains inconsistent formats such as numeric ("1", "2") and alphabetical ("A", "B") representations. To standardize, alphabetical values have been replaced with their numeric equivalents: "A" → "1" and "B" → "2".

### 2. 2. De-duplication

In [118]:
# Create 'section' column: take non-blank value from section_x or section_y
merged_df["Section"] = merged_df.apply(
    lambda row: row["Section_x"] if pd.notna(row["Section_x"]) and str(row["Section_x"]).strip() != "" 
    else (row["Section_y"] if pd.notna(row["Section_y"]) and str(row["Section_y"]).strip() != "" 
          else ""),
    axis=1
)

In [119]:
# Normalize Part of Term: A → 1, B → 2, keep digits as-is
merged_df['Part of Term'] = merged_df['Part of Term'].astype(str).str.strip().str.upper()
merged_df['Part of Term'] = merged_df['Part of Term'].replace({'A': '1', 'B': '2'})
merged_df['Part of Term'] = pd.to_numeric(merged_df['Part of Term'], errors='coerce').astype('Int64')
print("Unique values in part_of_term after standardization:", merged_df['Part of Term'].dropna().unique())


Unique values in part_of_term after standardization: <IntegerArray>
[1, 2]
Length: 2, dtype: Int64


In [120]:
merged_df = merged_df.rename(columns={'Section': 'Subject/Section', 'Subject_y': 'Subject', 'Number': 'Course_Number', 'Instructors': 'Instructors_Abbr', 'Primary Instructor': 'Instructors_FN', 'Type':'Section Type', 'Type Code':'Section Type Code'})
merged_df = merged_df.drop(columns = ['Subject_x', 'YearTerm', 'source_file', 'Sched Type', 'Course ', 'Course', 'Name', 'Status Code', 'Section_x', 'Section_y' ], axis=1)

### 2. 3. Reordering Columns

In [121]:
merged_df = merged_df[['Year', 'Term', 'CRN', 'Subject/Section', 'Section Title', 'Course_Number', 'Course Title', 'Description', 'Part of Term', 
                       'Degree Attributes', 'Credit Hours', 'Section Credit Hours', 'Section Status', 'Section Type', 'Section Type Code',  
                       'Enrollment Status', 'Start Time', 'End Time', 'Days of Week', 'Room', 'Building', 'Instructors_Abbr', 'Instructors_FN',
                       'Section Info', 'Schedule Information', 'Average Grade',
                       'A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F', 'W', 
                       'A Range', 'B Range', 'C Range', 'D Range']]

### 2. 4. Dropping High-Null Values

In [122]:
missing_percent = merged_df.isnull().mean() * 100
print("\nTop 20 columns by missing percentage:")
print(missing_percent.sort_values(ascending=False).head(20))
merged_df.drop(columns=missing_percent[missing_percent > 90].index, inplace=True)


Top 20 columns by missing percentage:
A Range                 100.000000
B Range                 100.000000
C Range                 100.000000
D Range                 100.000000
Section Title            93.319030
Schedule Information     83.444766
Section Credit Hours     77.083125
Degree Attributes        74.832851
Room                     38.434288
Building                 38.434288
Section Info             20.232512
End Time                 14.469614
Days of Week             14.245085
Part of Term              3.268137
Instructors_Abbr          0.239497
Instructors_FN            0.184612
Year                      0.000000
Term                      0.000000
CRN                       0.000000
Subject/Section           0.000000
dtype: float64


### 2. 5. Remove Duplicates

In [123]:
initial_rows = merged_df.shape[0]
merged_df.drop_duplicates(inplace=True)
removed = initial_rows - merged_df.shape[0]
print(f"Removed {removed} duplicate rows.")

Removed 77 duplicate rows.


### 2. 6. Handling Missing Values

In [124]:
# Fill text fields with 'Unknown'
text_cols = merged_df.select_dtypes(include='object').columns
merged_df[text_cols] = merged_df[text_cols].fillna('Unknown')

# Fill numeric grades with 0
grade_cols = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F', 'W', 'Average Grade']
for col in grade_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)


### 2. 7. Handle outliers

In [125]:
# Clip all grades to a maximum of 100%
for col in grade_cols:
    merged_df[col] = merged_df[col].clip(lower=0, upper=100)

### 2. 8. Cleaning Column Names

In [126]:
# Standardize column names: lowercase, underscores, no extra characters
merged_df.columns = (
    merged_df.columns.str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('+', '_plus')
    .str.replace('-', '_minus')
    .str.replace('/', '_')
)

print(merged_df.columns)

Index(['year', 'term', 'crn', 'subject_section', 'course_number',
       'course_title', 'description', 'part_of_term', 'degree_attributes',
       'credit_hours', 'section_credit_hours', 'section_status',
       'section_type', 'section_type_code', 'enrollment_status', 'start_time',
       'end_time', 'days_of_week', 'room', 'building', 'instructors_abbr',
       'instructors_fn', 'section_info', 'schedule_information',
       'average_grade', 'a_plus', 'a', 'a_minus', 'b_plus', 'b', 'b_minus',
       'c_plus', 'c', 'c_minus', 'd_plus', 'd', 'd_minus', 'f', 'w'],
      dtype='object')


### 2. 9. Fix and impute credit hours

In [127]:
# Standardize and fill missing credit hours
merged_df['credit_hours'] = merged_df['credit_hours'].replace(['nan', 'Unknown'], pd.NA)
merged_df['credit_hours'] = pd.to_numeric(merged_df['credit_hours'], errors='coerce')

# Impute using mode per (subject_section, course_number)
credit_mode = (
    merged_df.groupby(['subject_section','course_number'])['credit_hours']
    .agg(lambda x: x.dropna().mode().iloc[0] if not x.dropna().mode().empty else pd.NA)
)
merged_df['credit_hours'] = merged_df.apply(
    lambda row: credit_mode.get((row['subject_section'], row['course_number']), row['credit_hours']), axis=1
)


### 2. 10. Final sanity checks and export

In [128]:
# Drop any rows with missing course_number or year

merged = merged_df[merged_df['course_number'].notna() & merged_df['year'].notna()]
print(f"Dropped {before - merged_df.shape[0]} rows missing key course fields")

# Save cleaned dataset
merged.to_csv("merged_cleaned_final.csv", index=False)
print("Saved cleaned dataset")

Dropped 77 rows missing key course fields
Saved cleaned dataset
